In [33]:
from bs4 import BeautifulSoup as bs
from bs4 import SoupStrainer
from selenium import webdriver
import time, re
import requests
from itertools import repeat
import pandas as pd
import numpy as np
import unicodedata2
from tqdm import tqdm 

In [34]:
link_list = []
unique_id = []
speech_type = []

In [35]:
# Faudrait faire une liste et startwith tuple(list) mais après comment j'append ? 
types_de_speech = ['Déclaration','Interview','Allocution','Conférence de presse',"Extraits d'un entretien",'Entretien','Message']
for i in tqdm(range(51,800)):
    link = 'https://www.vie-publique.fr/discours' +'?page='+ str(i)
    page_resp=requests.get(link)
    content= bs(page_resp.content,'html.parser',parse_only=SoupStrainer('main'))
    for link in content.find_all("a", class_='link-multiple'):
        if link.find('span').text.startswith(tuple(types_de_speech)):
            link_list.append('https://www.vie-publique.fr'+ link.get('href'))
            unique_id.append(link.get('href').split('/')[2].split('-')[0])
            speech_type.append(link.find('span').text.split(' ')[0])

100%|██████████| 749/749 [09:41<00:00,  1.29it/s]


Essai sur les premières pages 

In [39]:
LINKS=pd.DataFrame({'unique_id':unique_id,'speech_type':speech_type,'link':link_list})
LINKS=LINKS.drop_duplicates(subset=['unique_id'],ignore_index=True)
LINKS.head()

,unique_id,speech_type,link
0,272458,Déclaration,https://www.vie-publique.fr/discours/272458-br...
1,272635,Déclaration,https://www.vie-publique.fr/discours/272635-je...
2,272488,Déclarations,https://www.vie-publique.fr/discours/272488-br...
3,272186,Extraits,https://www.vie-publique.fr/discours/272186-en...
4,272235,Interview,https://www.vie-publique.fr/discours/272235-je...


In [40]:
LINKS

,unique_id,speech_type,link
0,272458,Déclaration,https://www.vie-publique.fr/discours/272458-br...
1,272635,Déclaration,https://www.vie-publique.fr/discours/272635-je...
2,272488,Déclarations,https://www.vie-publique.fr/discours/272488-br...
3,272186,Extraits,https://www.vie-publique.fr/discours/272186-en...
4,272235,Interview,https://www.vie-publique.fr/discours/272235-je...
...,...,...,...
7294,200488,Déclaration,https://www.vie-publique.fr/discours/200488-de...
7295,200416,Déclaration,https://www.vie-publique.fr/discours/200416-de...
7296,200320,Déclaration,https://www.vie-publique.fr/discours/200320-de...
7297,200407,Déclaration,https://www.vie-publique.fr/discours/200407-de...


In [41]:
length = '0-800-2902020'
LINKS.to_csv('DATA_' +length+'.csv')

Quelles infos extra prendre ? 
- Intervenant (PB quand plusieurs si non journaliste ? ) 
- Fonction (OK mais possibles erreurs à checker ! ) 
- Titre 
- date # mettre as date 
- sujet principal (about) # pas encore fait
- Thématique 
- publications dans d'autres médias ? # pas encore fait
- Interview : mettre radio ou journal considéré
- discours ( à checker)

Ex : pb avec https://www.vie-publique.fr/discours/272865-ministere-de-leurope-et-des-affaires-etrangeres-20012020-ormuz 

In [43]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
#chrome_options.add_argument('--no-sandbox') 
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument("--disable-extensions")

# Selenium

In [ ]:
# Premier code d'extraction, deuxième code après le premier round pour ceux qui ont pas marché. 
#print('DATA_' +length)
from selenium.common.exceptions import NoSuchElementException
prenom=[]
nom=[]
titre=[]
date=[]
theme=[]
mots_cle=[]
discours=[]
merge_id=[]
fonct=[]
inter_fonct=[]

d = webdriver.Chrome(options=chrome_options)

for j in tqdm(range(len(LINKS.link))):
    try:
        d.get(LINKS.link[j])
    except:
        pass
    
    merge_id.append(LINKS.unique_id[j])
    try:
        titre.append(d.find_element_by_xpath("//h1[contains(@class, 'h')]").text)
    except:
        titre.append(np.nan)
    try:
        fonct.append(d.find_element_by_class_name("line-intervenant").text.replace('\n',' ').split(' - ')[1])  
    except:
        fonct.append(np.nan)       
    try:
        theme.append(d.find_element_by_class_name('thematicBox').text.replace('Thématique(s) :\n',''))
    except:
        theme.append(np.nan)
    try:
        prenom.append(d.find_element_by_class_name("line-intervenant").text.split(' - ')[0].split(' ')[0])
    except: 
        prenom.append(np.nan)
    try:
        nom.append(' '.join(d.find_element_by_class_name("line-intervenant").text.split('\n')[0].split(' - ')[0].split(' ')[1:]))
    except:
        nom.append(np.nan)
    try:
        date.append(d.find_element_by_tag_name('time').get_attribute(("datetime"))) # set as date 
    except:
        date.append(np.nan)
    try:
        mots_cle.append(d.find_element_by_class_name('tagsBox').text.replace('MOTS-CLÉS :\n','').replace('\n',' '))
    except:
        mots_cle.append('PAS DE MOTS CLE')
    try: 
        para=d.find_element_by_xpath("//span[contains(@class, 'texte-integral')]").find_element_by_xpath("//span[contains(@class, 'texte-integral')]").text.split('\n')
        para = [x for x in para if x != '']
        discours_s= []
        # A rassembler les if en un seul quand même code avec liste et isin 
        if LINKS.speech_type[j]=='Déclaration': 
            for a in para:
                if not a.startswith('Source'):  
                    discours_s.append(a)
                    #merge_id_s.append(unique_id[j])
                    
        if LINKS.speech_type[j]=='Allocution':
            for a in para:
                if not a.startswith('Source'):  # remplacer par html mais prend temps
                    discours_s.append(a)
                    #merge_id_s.append(unique_id[j])
                    
        if LINKS.speech_type[j]=='Message':
            for a in para:
                if not a.startswith('Source'):  # remplacer par html mais prend temps
                    discours_s.append(a)
                    #merge_id_s.append(unique_id[j])
                    
        if LINKS.speech_type[j]=='Interview':
            for i in range(len(para)-1):
                nomprenom=d.find_element_by_class_name("line-intervenant").text.split(' - ')[0].upper()
                if para[i].startswith(nomprenom):
                    if not para[i].replace(nomprenom,'')==' ':
                        discours_s.append(para[i].replace(nomprenom,''))
                    if not para[i+1].replace(nomprenom,'')==' ':
                        discours_s.append(para[i+1].replace(nomprenom,''))
            if len(discours_s)==0:
                for i in range(len(para)-1):
                    if para[i].startswith('R -'):
                        discours_s.append(para[i].replace('R -',''))
            if len(discours_s)==0:
                prenom_nom=d.find_element_by_class_name("line-intervenant").text.split(' - ')[0].split(' ')[0] + ' '+ ' '.join(d.find_element_by_class_name("line-intervenant").text.split('\n')[0].split(' - ')[0].split(' ')[1:]) +' :'
                for i in range(len(para)-1):
                    if para[i].startswith(prenom_nom):
                        discours_s.append(para[i].strip(prenom_nom))
            if len(discours_s)==0:
                if para[0].startswith('?'):
                    for i in range(len(para)-1):
                        if not para[i].startswith('?'):
                            discours_s.append(para[i])
                        
        if LINKS.speech_type[j]=='Entretien':
            for i in range(len(para)-1):
                if para[i].startswith(d.find_element_by_class_name("line-intervenant").text.split(' - ')[0].upper()):
                    discours_s.append(para[i+1])
            if len(discours_s)==0:
                for i in range(len(para)-1):
                    if para[i].startswith('R -'):
                        discours_s.append(para[i].replace('R -','')) # Parfois relou ca prend deux paragraphes... faut voir pour stopper quand Q - + je peux pas faire if not
# Un exemple : https://www.vie-publique.fr/discours/185845-entretien-de-m-laurent-fabius-ministre-des-affaires-etrangeres-avec-l

        if LINKS.speech_type[j]=='Extraits':
            for i in range(len(para)-1):
                if para[i].startswith(d.find_element_by_class_name("line-intervenant").text.split(' - ')[0].upper()):
                    discours_s.append(para[i+1])
            if len(discours_s)==0:
                for i in range(len(para)-1):
                    if para[i].startswith('R -'):
                        discours_s.append(para[i].replace('R -',''))
                        
        if LINKS.speech_type[j]=='Conférence':
            for i in range(len(para)-1):
                if para[i].startswith(d.find_element_by_class_name("line-intervenant").text.split(' - ')[0].upper()):
                    discours_s.append(para[i+1])
            if len(discours_s)==0:
                for i in range(len(para)-1):
                    if para[i].startswith('R -'):
                        discours_s.append(para[i].replace('R -',''))
        discours.append(' '.join(discours_s))
    except:
        discours.append(np.nan)
    #time.sleep(1)
    #print(j)
d.close()
print(j)

# NB : Peut être essayer de récupérer le texte même si erreur ? ( on en a assez en vrai)
# J'étais obligée les try except pour chaque truc ca bug à chaque fois sinon ... Autre méthode ? 
                                      
# Plus je pense que la ca fonctionne jusquà la page 1000, j'ai un soucis de nom aussi 
                                      

 71%|███████   | 5173/7299 [16:03:49<5:29:45,  9.31s/it]       

# Beautiful SOup

In [10]:
DATA =pd.DataFrame({'titre':titre,
                    'link':LINKS.link,
                    'speech_type' :LINKS.speech_type,
                    'prenom':prenom,
                    'nom' : nom,
                    'date':date,
                   'theme':theme,
                   'fonct':fonct,
                   'unique_id':LINKS.unique_id}
                  )
DISCOURS = pd.DataFrame({'merge_id':merge_id,
                         'discours':discours})


In [11]:
DATA.to_csv('DATA.csv')
DISCOURS.to_csv('DISCOURS.csv')

In [23]:
DATA=pd.read_csv('DATA.csv')
DISCOURS=pd.read_csv('DISCOURS.csv')

In [12]:
for i in range(len(mots_cle)):
    if mots_cle[i] is np.nan:
        mots_cle[i]=''
    else:
        mots_cle[i]=mots_cle[i].replace(' - ','-')
        mots_cle[i]=mots_cle[i].replace("INSTITUTIONS DE L'ETAT","INSTITUTIONS-DE-L'ETAT")
        mots_cle[i]=mots_cle[i].replace(' EUROP','-EUROP')
        mots_cle[i]=mots_cle[i].replace('VIE EC','VIE-EC')
        mots_cle[i]=mots_cle[i].replace('ITIQUE ','ITIQUE-')
        mots_cle[i]=mots_cle[i].replace('ZONE ','ZONE-')
        mots_cle[i]=mots_cle[i].replace('RELATIONS ','RELATIONS-')
        mots_cle[i]=mots_cle[i].split(' ') 
# Faire une sorte de regroupement de mots cle avec l'algo vu en cours ? 


In [13]:
col= []
for i in range(len(max([x for x in mots_cle if x is not np.nan], key=len))):
    col.append('MC'+str(i+1))
col

['MC1', 'MC2', 'MC3', 'MC4', 'MC5', 'MC6', 'MC7', 'MC8']

In [14]:
MOTS_CLE=pd.DataFrame(data=mots_cle,columns=col)
MOTS_CLE.to_csv('MOTS_CLE.csv')

In [15]:
DATA=pd.concat([DATA,MOTS_CLE],axis=1)
DATA=DATA.merge(DISCOURS, left_on='unique_id',right_on='merge_id')
DATA.columns
DATA.head()

,titre,link,speech_type,prenom,nom,date,theme,fonct,unique_id,MC1,MC2,MC3,MC4,MC5,MC6,MC7,MC8,merge_id,discours
0,"Interview de M. Franck Riester, ministre de la...",https://www.vie-publique.fr/discours/273637-fr...,Interview,Franck,Riester,2020-02-28T12:00:00Z,SOCIÉTÉ,Ministre de la culture,273637,CULTURE-MÉDIAS,CINÉMA,TÉLÉVISION,None,None,None,None,None,273637,Bonjour Marc FAUVELLE. Oui bien sûr ! Oui a...
1,"Déclaration de M. Franck Riester, ministre de ...",https://www.vie-publique.fr/discours/273623-fr...,Déclaration,Franck,Riester,2020-02-26T12:00:00Z,SOCIÉTÉ,Ministre de la culture,273623,CULTURE-MÉDIAS,MÉDIA,None,None,None,None,None,None,273623,"Merci monsieur le président, cher Bruno Studer..."
2,"Interview de M. Bruno Le Maire, ministre de l'...",https://www.vie-publique.fr/discours/273614-br...,Interview,Bruno,Le Maire,2020-02-26T12:00:00Z,ECONOMIE,Ministre de l'économie et des finances,273614,VIE,ÉCONOMIQUE,POLITIQUE-ÉCONOMIQUE,None,None,None,None,None,273614,Bonjour. Ça c'est la prérogative du Premier ...
3,"Déclaration de M. Jean-Yves Le Drian, ministre...",https://www.vie-publique.fr/discours/273609-je...,Déclaration,Jean-Yves,Le Drian,2020-02-24T12:00:00Z,INSTITUTIONS,Ministre de l'Europe et des affaires étrangères,273609,JUSTICE-DROITS,FONDAMENTAUX,DROITS,DE,L'HOMME,None,None,None,273609,Madame la Présidente du Conseil des droits de ...
4,"Interview de Mme Christelle Dubos, secrétaire ...",https://www.vie-publique.fr/discours/273645-ch...,Interview,Christelle,Dubos,2020-02-24T12:00:00Z,SOCIÉTÉ,Secrétaire d'État à la santé,273645,SANTÉ-PROTECTION,SOCIALE,MALADIE,None,None,None,None,None,273645,Bonjour. Oui en effet la personne est soigné...


In [16]:
col=[col for col in DATA if col.startswith('MC')]

In [17]:
for C in col : 
    DATA[C]=DATA[C].replace(to_replace=[None], value=' ')

In [18]:
DATA=DATA[~DATA.prenom.isna()]

In [19]:
DATA=DATA[~DATA.fonct.isna()]

In [20]:
# Pour les discours plus vieux, la fonction se trouve uniquement dans le titre 
#DATA['fonct'] = DATA.apply(
#    lambda row: row['titre'].split(',')[1] if np.isnan(row['fonct']) else row['fonct'],
#    axis=1

#)

In [21]:
DATA.dropna(inplace=True)

In [22]:
DATA=DATA[DATA.discours != '']
DATA.reset_index(inplace=True)
del DATA['index']
del DATA['level_0']

KeyError: 'level_0'

In [23]:
DATA.to_csv('Sample_0_200.csv')

In [24]:
len(DATA)

355

In [25]:
#Faut voir avec le merge id si ca colle ... 

### Merge label prénoms

In [26]:
# On peut aussi aller chercher le madame monsieur ! 
prenoms=pd.read_csv(r'C:\Users\morga\Documents\GitHub\NLP_brouillon\nat2018.csv',sep=';')
prenoms=prenoms.groupby(['preusuel','sexe'],as_index=False).sum()
prenoms=prenoms.sort_values(['nombre'],ascending=True)
prenoms=prenoms.drop_duplicates(subset=['preusuel'],keep='last') # On prend le sexe qui apparait le plus 
prenoms=prenoms[['preusuel','sexe']]
DATA.prenom=[pre.upper() for pre in DATA.prenom.astype('string')]
DATA=DATA.merge(prenoms, left_on='prenom',right_on='preusuel')

In [27]:
DATA.to_csv('Sample_0_200.csv')

### Themes 

In [28]:
DATA=pd.read_csv('Sample_0_200.csv')

In [29]:
DATA=DATA.drop(columns='preusuel')

In [30]:
theme_clean=[]
for theme in DATA.theme:
    if len(theme.split('\n'))==0:
        theme_clean.append(theme)
    if len(theme.split('\n'))>0:
        theme_clean.append(theme.split('\n')[0])
    

In [31]:
DATA.theme=theme_clean

In [32]:
DATA

,Unnamed: 0,titre,link,speech_type,prenom,nom,date,theme,fonct,unique_id,...,MC2,MC3,MC4,MC5,MC6,MC7,MC8,merge_id,discours,sexe
0,0,"Interview de M. Franck Riester, ministre de la...",https://www.vie-publique.fr/discours/273637-fr...,Interview,FRANCK,Riester,2020-02-28T12:00:00Z,SOCIÉTÉ,Ministre de la culture,273637,...,CINÉMA,TÉLÉVISION,,,,,,273637,Bonjour Marc FAUVELLE. Oui bien sûr ! Oui a...,1
1,1,"Déclaration de M. Franck Riester, ministre de ...",https://www.vie-publique.fr/discours/273623-fr...,Déclaration,FRANCK,Riester,2020-02-26T12:00:00Z,SOCIÉTÉ,Ministre de la culture,273623,...,MÉDIA,,,,,,,273623,"Merci monsieur le président, cher Bruno Studer...",1
2,2,"Interview de M. Franck Riester, ministre de la...",https://www.vie-publique.fr/discours/273513-fr...,Interview,FRANCK,Riester,2020-02-19T12:00:00Z,SOCIÉTÉ,Ministre de la culture,273513,...,A.,ECONOMIE,NUMÉRIQUE,JUSTICE-DROITS,FONDAMENTAUX,LIBERTÉS,INDIVIDUELLES,273513,"Bonjour. Oui oui, et donc le président de la...",1
3,3,"Déclaration de M. Franck Riester, ministre de ...",https://www.vie-publique.fr/discours/273477-fr...,Déclaration,FRANCK,Riester,2020-02-18T12:00:00Z,SOCIÉTÉ,Ministre de la culture,273477,...,PROFESSION,ARTISTIQUE,,,,,,273477,"Mesdames et Messieurs les Parlementaires, Mesd...",1
4,4,"Déclaration de M. Franck Riester, ministre de ...",https://www.vie-publique.fr/discours/273622-fr...,Déclaration,FRANCK,Riester,2020-02-18T12:00:00Z,SOCIÉTÉ,Ministre de la culture,273622,...,A.,ECONOMIE,NUMÉRIQUE,,,,,273622,C'est un vrai plaisir que d'être cet après-mid...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,342,"Interview de M. Jean-Michel Blanquer, ministre...",https://www.vie-publique.fr/discours/272645-je...,Interview,JEAN-MICHEL,Blanquer,2019-12-17T12:00:00Z,SOCIÉTÉ,Ministre de l'éducation nationale et de la jeu...,272645,...,SOCIALE,RETRAITE,EDUCATION-RECHERCHE,ENSEIGNANT,,,,272645,"Ça doit être 25 % dans le premier degré, puis...",1
343,343,"Interview de M. Jean-Michel Blanquer, ministre...",https://www.vie-publique.fr/discours/272640-je...,Interview,JEAN-MICHEL,Blanquer,2019-12-12T12:00:00Z,SOCIÉTÉ,Ministre de l'éducation nationale et de la jeu...,272640,...,SOCIALE,RETRAITE,,,,,,272640,Bonjour Marc FAUVELLE. Je ne pense pas que n...,1
344,344,"Interview de M. Jean-Michel Blanquer, ministre...",https://www.vie-publique.fr/discours/272647-je...,Interview,JEAN-MICHEL,Blanquer,2019-12-10T12:00:00Z,SOCIÉTÉ,Ministre de l'éducation nationale et de la jeu...,272647,...,SOCIALE,RETRAITE,,,,,,272647,"Pour le premier degré, on est capable de don...",1
345,345,"Interview de M. Jean-Michel Blanquer, ministre...",https://www.vie-publique.fr/discours/272649-je...,Interview,JEAN-MICHEL,Blanquer,2019-12-06T12:00:00Z,SOCIÉTÉ,Ministre de l'éducation nationale et de la jeu...,272649,...,SOCIALE,RETRAITE,,,,,,272649,Bonjour Jean-Jacques BOURDIN. C'est in...,1


In [21]:
DATA.to_csv('Sample_0_200.csv')